<h1> Use this <a href="http://nbviewer.jupyter.org/github/AdaTeamElec/ADA2017-Homeworks/blob/master/project/IntroAda.ipynb">link</a> to properly display maps </h1>

# Intro

## Abstract
Terrorism is a subject largely covered in the media, and, unfortunately, we became accustomed to its presence worldwide, particularly over the last decade. Nevertheless, the problem we are facing today is not new. The source of certain conflicts dates from multiple decades, some of which are still lasting today. Our goal is to track and vizualize terrorism evolution through the past 50 years based on "The Global Terrorism Database". There are many questions we can ask ourselves about terrorism, such as "Is EU less safe nowadays ?", "Did attack mediums & reasons change over the years ?" or "Can we discriminate current/future conflictual zones ?". It would be presumptuous from us to say that we are going to solve major issues, or even predict futur attacks. However, through the exploration of the dataset, and by trying to answer those interrogations, we aim to grasp an overview and a better understanding to the evolution of terrorism.

## Plan

1. [Raw data understanding and cleaning](#raw_data)
    1. [Field selections using documentation](#fields_select)
    2. [Data exploration ](#data_exploration)
2. [Groups](#groups)
3. [Groups territories](#groups_terr)

---

# 1 Raw data understanding  <a id='raw_data'></a>

## 1.1 Field selections using documentation  <a id='fields_select'></a>

First of all, we need to take a deep look into the details of our dataset to sort out the relevant data we will be using to conduct our observations. The Global Terrorism Dataset contains 135 features and approximately 170'000 entries. In order to select the label we will keep, we used the official [documentation](http://start.umd.edu/gtd/downloads/Codebook.pdf) from the dataset which describes each features precisely. Let's make a quick summary of the labels from the dataset we decided to use for our project.

* `eventid` : this is the id of any entry, written as 12 numbers (first 8 digits are the date of event and last 4 digits are a sequential case number for the given day). This will be used as our index too.
* `iyear`, `imonth`, `iday` : Year, month and day of the event. In some rare occasion the month or days are unknown.
* `country_txt` : id and name of the country where the event took place.
* `region_txt` : id and region where the event took place.
* `city` : This field contains the name of the city, village, or town in which the incident occurred. If the city, village, or town for an incident is unknown, then this field contains the smallest administrative area below provstate which can be found for the incident (e.g., district).  
* `latitude` and `longitude` : Latitude and Longitude values where the event took place.
* `doubtterr` : boolean value set as 1 if there is a doubt to whether the incident is an act of terrorism and 0 if there is no doubt of a terrorist attack.
* `success` : boolean value set as 1 if the incident was successful or 0 if it was not. As stated in the documentation, "Success of a terrorist strike is defined according to the tangible effects of the attack. Success is not judged in terms of the larger goals of the perpetrators. For example, a bomb that exploded in a building would be counted as a success even if it did not succeed in bringing the building down or inducing government repression." 
* `suicide` : boolean value set as 1 if the attack perpetrator did not intend to escape from the attack alive, 0 otherwise.
* `attacktype1_txt` : This field captures the general method of attack and often reflects the broad class of tactics used. It consists of nine categories, which are defined below :
    1. Assassination
    2. Armed Assault
    3. Bombing/Explosion
    4. Hijacking 
    5. Hostage taking (barricade incident) 
    6. Hostage taking (kidnapping)
    7. Facility/Infrastructure Attack
    8. Unarmed Assault
    9. Unknown 
* `targtype1_txt` : The target/victim type field captures the general type of target/victim. When a victim is attacked specifically because of his or her relationship to a particular person, such as a prominent figure, the target type reflects that motive. For example, if a family member of a government official is attacked because of his or her relationship to that individual, the type of target is “government.” This variable consists of the following 22 categories: <br>
    1. Business
    2. Government (General)
    3. Police
    4. Military
    5. Abortion related
    6. Airport & aircraft
    7. Government (Diplomatic), differs from the other entry as here are taken into account representation of a gouvernment on a foreign soil (embassy, consulate...)
    8. Educational institution
    9. Food or water supply
    10. Journalist & media
    11. Maritime facilities, including ports
    12. NGO
    13. Other
    14. Private citizens & property, include attacks in a public area against private citizens
    15. Religious figures/insititutions
    16. Telecommunication
    17. Terrorists/non-state militias
    18. Tourists
    19. Transportation (other than aviation)
    20. Unknown
    21. Utilities, facilities for generation or transmission of energy
    22. Violent political parties
* `gname` : This field contains the name of the group that carried out the attack. In order to ensure consistency in the usage of group names for the database, the GTD database uses a standardized list of group names that have been established by project staff to serve as a reference for all subsequent entries.  
* `gname2` : This field is used to record the name of the second perpetrator when responsibility for the attack is attributed to more than one perpetrator. Conventions follow “Perpetrator Group” field.  
* `gname3` : same as for gname2
* `nperps` : This field indicates the total number of terrorists participating in the incident. (In the instance of multiple perpetrator groups participating in one case, the total number of perpetrators, across groups, is recorded). There are often discrepancies in information on this value.   
* `weaptype1_txt` : This field records the general type of weapon used in the incident. It consists of the following categories: <br>
    1. Biological
    2. Chemical
    3. Radiological
    4. Nuclear
    5. Firearms
    6. Explosive/bonbs/dynamite
    7. Fake weapons
    8. Incendiary
    9. Melee
    10. Vehicle
    11. Sabotage equipment 
    12. Other
    13. Unknown
* `nkill` : This field stores the number of total confirmed fatalities for the incident. The number includes all victims and attackers who died as a direct result of the incident.   
* `nkillter`: This field stores the number of confirmed terrorists fatalities.
* `nwound` : This field records the number of confirmed non-fatal injuries to both perpetrators and victims. 
* `nwoundte` : This field records the number of confirmed non-fatal terrorists injuries. 


We are now reduced to 22 features instead of the original 135 from the dataset. A part from the kept features, we explored some other features such as `weaptype2`, `weapsubtype` or `motive` to see if those would bring added informations thus be relevant to use also. However, we decided to drop them because of a too large amount of NaN or unknown entries. Our choice focused on labels that would allow us to answer the questions asked in the description, as well as labels relevant to get a pertinent visualization of the data.


## 1.2 Data exploration  <a id='data_exploration'></a>

Let's begin the work by importing the libraries and creating a dataframe to explore the data furthermore. As cautious wannabe data scientist, we will explore in detail each field and check the proportion of non categorized or Unknown-labeled entries to make sure each feature we kept countains relevant data. <br>
*NOTE : during this section we are only exploring data without drawing any conclusions nor making assumptions regarding the data. This will come further in our analysis*

In [ ]:
import pandas as pd
import os
import numpy as np
import datetime
import time
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import warnings

%load_ext autoreload
%autoreload 2

%pylab inline
%matplotlib inline

sns.set_context("notebook")
warnings.filterwarnings('ignore')

In [ ]:
data_path = 'data'
gtd_path = os.path.join(data_path, 'globalterrorismdb_0617dist.csv')

In [ ]:
fields = ['eventid', 'iyear', 'imonth', 'iday', 'country_txt', 'region_txt', 'city', 
          'latitude', 'longitude', 'doubtterr', 'attacktype1_txt',  'success', 
          'suicide', 'weaptype1_txt', 'targtype1_txt', 'gname', 'gname2', 
          'gname3', 'compclaim', 'nperps', 'nkill', 'nkillter', 'nwound', 'nwoundte']
date_fileds = ['iyear', 'imonth', 'iday']

df = pd.read_csv(gtd_path, encoding='latin', usecols=fields, index_col='eventid', low_memory=False)

In [ ]:
print('Is index unique: {}'.format(df.index.is_unique))

In [ ]:
df.dtypes

According to the documentation, month or day (or both) can be set to 0 if the exact date of the attack is unknown. We created a function to set the value to of the field to 1 in the case of an unknown date. We then count the proportion of unknown date within the dataset, just to make sure it is not too high.

In [ ]:
# According to documentation both month and day can be 0 (if unknown), we set them to 0
def parse_date(row):
    return datetime.date(row.iyear, int(row.imonth) if not np.isnan(row.imonth) else 1, 
                         int(row.iday) if not np.isnan(row.iday) else 1)

In [ ]:
# Count number entries with uncertain date (either month or day)
df[date_fileds] = df[date_fileds].replace(0, np.nan)
n_uncertain = np.sum(np.sum(df[date_fileds].isnull(), axis=1) != 0 )
df['date'] =df.apply(lambda x: parse_date(x), axis=1)
print('Uncertain dates: {:.2f}%, ({}/{})'.format(100*n_uncertain/len(df), n_uncertain, len(df)))

We check the proportion of entries without geographic coordinates. In the case when the coordinates are unknow, we decided to completely drop the row of data. This is due to the fact that we want to have the location informations in order to represent the data with maps.

In [ ]:
n_geo = len(df)
df.dropna(subset=('latitude', 'longitude'), inplace=True)
print('Entries without geographic coordinates droped: {:.2f}%, ({}/{})'.format(
    100*(n_geo-len(df))/n_geo, n_geo-len(df), n_geo))

We check that each item has a valid entry for city, country or region. We see that it is missing a few entries for city name, but as we will not use this feature often, we won't drop those entries.

In [ ]:
print('Proportion of data without a valid city entry: {:.2f}%'.format(100*(1-df.city.value_counts().sum()/len(df))))
print('Proportion of data without a valid country entry: {:.2f}%'.format(100*(1-df.country_txt.value_counts().sum()/len(df))))
print('Proportion of data without a valid region entry: {:.2f}%'.format(100*(1-df.region_txt.value_counts().sum()/len(df))))

We are now checking the amount of attacks that are categorized as unsure terror attacks. We wanted to check this particular feature to make sure that in the dataset there is a large majority of attacks that are hundred percent sure to be terror attack. If it would not have been the case, the whole dataset as well as our study would not have been relevant. <br>
There is 15% of attacks for which there is a doubt to categorize them as terror attacks. The entry -9 represent cases for which the value was not available at all when the dataset was constructed. We decided to assign them as if it was sure they were terror attacks.

In [ ]:
print('Repartition of data in %:\n{}'.format(100*df.doubtterr.value_counts()/len(df)))
df.loc[df.doubtterr < 0, 'doubtterr'] = 0
df.doubtterr = df.doubtterr.astype('category')
df.doubtterr.cat.categories = ['N_DOUBT', 'DOUBT']
print('\nRepartition of data in %(after cleaning):\n{}'.format(100*df.doubtterr.value_counts()/len(df)))

For the two upcoming fields, data is, as expected, binary and completly categorized.

In [ ]:
print('Unique values in field: {}'.format(np.unique(df.success)))
print('Percentage of sucessful attacks: {:.2f}%'.format(100*df.success.mean()))

In [ ]:
print('Unique values in field: {}'.format(np.unique(df.suicide)))
print('Percentage of suicide attacks: {:.2f}%'.format(100*df.suicide.mean()))

Time to explore if the proportion of attack types and see if there are any NaN values.

In [ ]:
print('Type of attack and repartition in dataset in %')
100*df.attacktype1_txt.value_counts()/len(df)

We do the same for the repartion of target types and the repartion of weapon types.

In [ ]:
print('Type of target and repartition in dataset in %')
100*df.targtype1_txt.value_counts()/len(df)

In [ ]:
print('Repartition of weapon type in dataset in %')
100*df.weaptype1_txt.value_counts()/len(df)

We now check the number of unique entries that categorize the name of the group conducting the terror attacks. According to the documentation, a work as been done to standardize the entries within this field by using a specific list of group names established by project staff.

In [ ]:
pd.value_counts(df[['gname', 'gname2', 'gname3']].values.ravel('K')).head(10)

In [ ]:
df[['gname', 'gname2', 'gname3']] = df[['gname', 'gname2', 'gname3']].replace({'Unknown': np.nan})
n_group = len(pd.unique(df[['gname', 'gname2', 'gname3']].values.ravel('K')))
print('Number of unique group name: {}'.format(n_group))

We will now explore data with numerical values. First we look at field corresponding to the number of perpretrators of an attack. As expected, it countains a large amount of unknown entries as it is not easy to know how many perpetrators of an attack there was. We decided to keep this row anyway as to explore, if possible, the evolution of terror attack, and the number of perpetrators is a value that could give an insight to know this.

In [ ]:
df.loc[df.nperps < 0, 'nperps'] = np.nan
print('Percentage of entries with unknown # Perpretrators {:.2f}%'.format(100*np.sum(df.nperps.isnull())/len(df)))
print('Range of # Perpretrators: {} upto {}'.format(int(df.nperps.min()), int(df.nperps.max())))

We look now at the number of victims of terror attacks. As the dataset count the total number of fatalities, perpetrators included, we found it relevant to keep also the number of killed terrorists to conduct our analysis. Same logic applies for the number of wounded.

In [ ]:
df['nkillnter'] = df.nkill-df.nkillter.fillna(0)
df.loc[df.nkillnter < 0, 'nkillnter'] = 0

print('Range total # of victims: [{}, {}]'.format(df.nkill.min(), df.nkill.max()))
print('Range # of non terrorists victims: [{}, {}]'.format(df.nkillnter.min(), df.nkillnter.max()))
print('Range # of terrorists victims: [{}, {}]'.format(df.nkillter.min(), df.nkillter.max()))

In [ ]:
df['nwoundnter'] = df.nwound-df.nwoundte.fillna(0)
df.loc[df.nwoundnter < 0, 'nwoundnter'] = 0

print('Range total # of wounded: [{}, {}]'.format(df.nwound.min(), df.nwound.max()))
print('Range # of non terrorists wounded: [{}, {}]'.format(df.nwoundnter.min(), df.nwoundnter.max()))
print('Range # of terrorists wounded: [{}, {}]'.format(df.nwoundte.min(), df.nwoundte.max()))

Now we have a better understanding of our dataset and we are confident to have a dataset we can work with. We can go further in our analysis and do data visualization.

---

# 2. Groups  <a id='groups'></a>


We can see that we have for each group: the number of attacks (`frequ`), number of casualities (`nkill`), coordinates (`latitude`, `longitude`) and country (`country`).

In [ ]:
df_groups = pd.read_csv(os.path.join(data_path, 'groups_stats.csv'), index_col=0)
df_groups.head()

In [ ]:
N_min = 20
len(df_groups.loc[df_groups.frequ > N_min])

In [ ]:
# Create 2d table with for matching pair-wise groups
frames = [ df[['gname', 'gname2', 'compclaim']].values, 
           df[['gname', 'gname3', 'compclaim']].values, 
           df[['gname2', 'gname3', 'compclaim']].values ]
df_multi = pd.DataFrame(np.array(frames).reshape((-1, 3)))
df_multi.columns = ['gname', 'gname2', 'compclaim']
df_multi.compclaim = df_multi.compclaim.replace({-9: np.nan})
# Drop line with NaN (not relevant matching)
df_multi.dropna(subset=('gname', 'gname2', 'compclaim'), inplace=True)
print('Number of colaboration/comp:', len(df_multi))
df_multi.head()

In [ ]:
# Set attrbute value to 1
df_coop = df_multi.loc[df_multi.compclaim == 0].groupby(['gname', 'gname2']).size().reset_index(name='n_coop')
df_comp = df_multi.loc[df_multi.compclaim == 1].groupby(['gname', 'gname2']).size().reset_index(name='n_comp')
df_link = pd.merge(df_coop, df_comp, how='outer').fillna(0)
df_link.head()


## 2.1 Signed Graph clustering example

Based on [Spectral Analysis of Signed Graphs for Clustering, Prediction and Visualization](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.155.1809&rep=rep1&type=pdf)

In [ ]:
import utils.signedgraph as sg

W = sg.get_dummy()

plt.figure(figsize=(5, 5))
ax = plt.subplot(1,1,1)
sg.draw_graph(W, ax=ax)

In [ ]:
L = sg.compute_L(W, normalized=False)
cgt = sg.get_clusters(L, 3)

plt.figure(figsize=(5, 5))
ax = plt.subplot(1,1,1)
sg.draw_graph(W, cgt, ax=ax)


## 2.2 Graph clustering -> data

In [ ]:
from scipy.sparse import csc_matrix
import networkx as nx

groups = pd.unique(df_link[['gname', 'gname2']].values.ravel('K'))
d = dict(zip(groups, np.arange(len(groups))))

# Create Adjacency matrix with # of attack claimed together as weight between links (e.i. groups)
W_coop = csc_matrix((df_link['n_coop'], 
                (df_link['gname'].replace(d), df_link['gname2'].replace(d))), shape=(len(d), len(d)))
W_coop = 0.5*(W_coop.T+W_coop)

W_comp = csc_matrix((df_link['n_comp'], 
                (df_link['gname'].replace(d), df_link['gname2'].replace(d))), shape=(len(d), len(d)))
W_comp = 0.5*(W_comp.T+W_comp)

W = (W_coop - W_comp) 

In [ ]:
G = nx.from_scipy_sparse_matrix(W)

In [ ]:
groups_clustering = {}
clusters = [list(g) for g in nx.connected_components(G) if len(g) >= 8]

for i, cluster in enumerate(clusters):
    print('Cluster length: {}\nNames: {}\n'.format(len(cluster), groups[cluster]))
    # Extract adjacency matrix fpr specific cluster
    W = sg.get_w_signed(W_coop, W_comp, cluster, binary=False)
    # Compute Nomralized signed laplacian
    L = sg.compute_L(W, normalized=True)
    # Get estimate of number of clusters
    ncluster = sg.estimate_ncluster(W, L, plotloss=False)
    
    # Get estimated clustering
    est_cgt = sg.get_clusters(L, ncluster)
    # Save results as dictionary
    groups_clustering[i] = {'W': W, 'ids': cluster, 'ncluster': ncluster, 'est_cgt': est_cgt}

In [ ]:
plt.figure(figsize=(16, 12*len(groups_clustering)))

for i, item in enumerate(groups_clustering):
    ax = plt.subplot(len(groups_clustering), 1, i+1)
    ax.set_title('Clustered (N={}) and organized graph'.format(groups_clustering[item]['ncluster']), fontsize=14)
    sg.draw_graph(groups_clustering[item]['W'], 
                  groups_clustering[item]['est_cgt'], 
                  labels=groups[groups_clustering[item]['ids']], ax=ax, reorder=True, offset=4e-1)


---
# 3. Groups  Territories <a id='groups_terr'></a>

In [ ]:
import utils.heat_gsp as heat_gsp
from pygsp import graphs, filters

## 3.1 Graph and signals

In [ ]:
# Take only Middle East & North Africa points
df_ME = df.loc[df.region_txt == 'Middle East & North Africa']
df_ME['id_loc'] = df_ME.latitude.astype(str) + '_' + df_ME.longitude.astype(str)

In [ ]:
# Drop duplicates to build map
nodes = df_ME.drop_duplicates(subset='id_loc')
nodes = nodes.reset_index().reset_index()
nodes = nodes[['latitude', 'longitude', 'id_loc', 'index']].set_index('id_loc')
W = heat_gsp.knn_graph(nodes)
print('Are nodes unique: {}'.format(nodes.index.is_unique))
print('Adjacency matrix shape: {}'.format(W.shape))

In [ ]:
np.random.seed(0)
frequ_min = 10
cls_groups = pd.value_counts(df_ME[['gname', 'gname2', 'gname3']].values.flatten())
cls_groups = cls_groups[cls_groups.values > frequ_min].index.values
cls_groups = np.random.permutation(cls_groups)
print('Number of groups with at least {} attacks: {}'.format(frequ_min, len(cls_groups)))

In [ ]:
n_time_interval = 10
intervals = np.linspace(0, len(df_ME)-1, n_time_interval+1).astype(int)
dates = df_ME.iloc[intervals].date.values

In [ ]:
df_time_inveral = df_ME.loc[np.logical_and(df_ME.date >= dates[9], df_ME.date <= dates[10])]
signal = heat_gsp.get_signal_attack(df_time_inveral, nodes, cls_groups)

In [ ]:
G = graphs.Graph(W)
G.estimate_lmax()

In [ ]:
id_ISIS = np.where(cls_groups == 'Islamic State of Iraq and the Levant (ISIL)')[0][0]
heat_gsp.plot_map_group(nodes, signal[:, id_ISIS], title='Attacks prformed by {}'.format(cls_groups[id_ISIS]))

In [ ]:
g_heat = filters.Heat(G, tau=100)
signal_heat = g_heat.filter(signal, method='chebyshev')
heat_gsp.plot_map_group(nodes, signal_heat[:,id_ISIS], 
                        title='Attacks prformed by {} - Spread'.format(cls_groups[id_ISIS]))

In [ ]:
signal_heat_thresh = np.concatenate((5e-2*np.ones((signal_heat.shape[0], 1)), signal_heat), axis=1)
id_groups = np.argmax(signal_heat_thresh, axis=1)

In [ ]:
heat_gsp.plot_map_cls(nodes, id_groups, cls_groups, 'Groups repartition in late 2016')